In [ ]:
# import the necessary libraries
from bs4 import BeautifulSoup
import requests
import json
import re
import datetime
import easyocr
import cv2
import geopy.distance
import concurrent.futures
import pgeocode

In [ ]:
# this function handles the situation when the html is embedded within javascript
def javascript_html_parse(source_arg):
	"""Return the JSON object as a dictionary"""
	start = "<script>window.__PRELOADED_STATE__ = "
	end = "</script"
	x = source_arg[source_arg.find(start)+len(start):]
	x = x[:x.find(end)]
	x = json.loads(x)
	return x

In [ ]:
# Obtain all of the URL's that need to be scraped from 
# This is the first URL we start at, here we are collecting all of the URL's to be scraped from into 
# local area URL's 
url = "https://www.rightmove.co.uk/house-prices-in-London.html"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')
b = soup.find_all("a", class_="head")
london_urls = []
for i in b:
	london_urls.append(i["href"])

borough_urls = []
for url in london_urls:
	source = requests.get(url).text
	soup = BeautifulSoup(source, 'lxml')
	b = soup.find_all("a", class_="head")
	for i in b:
		borough_urls.append(i["href"])

local_area_urls = []
for url in borough_urls:
	source = requests.get(url).text
	soup = BeautifulSoup(source, 'lxml')
	b = soup.find_all("a", class_="head")
	for i in b:
		local_area_urls.append(i["href"])

In [ ]:
# iterate over all of the URL's and scrape from them, and store the scraped data into a variable called houses
m_values = ["square meters", "square metres", "square meter","square metre","square m","square mt","sqmt","sq mt","sq.mt", "sq. mt", "sqm", "sq m", "sq.m", "sq. m", "sq: m", "sq:m", "sq :m", "sq:  m", "sq : m", "meters2", "metres2", "meter2", "metre2", "mt2", "m2"]
f_values = ['square feet','square ft','square f','sqft','sq ft','sq.ft','sq. ft','sqf','sq f','sq.f','sq. f','sq: f','sq:f','sq :f','sq : f','sq :f','feet2','ft2','f2']
houses = {}
house_features = {}
#for area_url in local_area_urls[:10]:
def scrape_data(area_url): 

    # while loop that obtains the address, property type, last sold price, last sold date, and its url (if it has one) from each house. 
    # Stores each value into a list. This is from the web page which shows different houses from a particular area.

    page_number = 1
    addresses, property_type, last_sold_price, last_sold_date, url = [], [], [], [], []
    # obtain the number of pages of properties jsonto use in the while loop condition
    source = requests.get(area_url).text
    a = javascript_html_parse(source)
    num_pages = a["pagination"]["last"]
    
    while page_number <= num_pages:

        if page_number == 1:
            source = requests.get(area_url).text
        else:
            source = requests.get(area_url+f"?page={page_number}").text

        # Scrape the address, property type, the price & date of the last sale, and the URL.
        x = javascript_html_parse(source)
        for house in x["results"]["properties"]:
            clean_address = house["address"].replace("'", "")
            addresses.append(clean_address)
            print(clean_address)
            property_type.append(house["propertyType"])
            last_sold_price.append(house["transactions"][0]["displayPrice"])
            last_sold_date.append(house["transactions"][0]["dateSold"])
            url.append(house["detailUrl"])



        page_number += 1


    # Store the scraped data for each property into the houses dictionary
    for i in range(len(addresses)):
        houses[addresses[i]] = {"property_type":property_type[i], "price":last_sold_price[i], "date":last_sold_date[i], "url": url[i]}


    # Now loop through all of the stored URL's for each property, and scrape more data from it.

    for a, v in houses.items():
        if v['url'] == "":
            continue
        else:
            source = requests.get(v['url']).text
            soup = BeautifulSoup(source, 'lxml')
            script = soup.find("script", {"type":"text/javascript"}).text
            # Regex used to convert the JSON data structure into a python dict 
            # Use regex to extract json data from the script text
            json_script = re.findall(("(?s)(?<=window.PAGE_MODEL = )(.*$)"), script)[0]
            # Transforming json data within string into dictionary
            json_dict = json.loads(json_script)

            # The bullet pointed features - put this in a separate dictionary from all of the other data for now
            #try:
            #    features = json_dict["soldPropertyData"]["property"]["keyFeatures"]
            #except TypeError:
            #    pass
            #else:
            #    house_features[a] = features
            #    v["features"] = features
            # The floorplan URL
            try:
                floorplan_url = json_dict["soldPropertyData"]["property"]["floorplans"][0]['url']
            except Exception:
                # seen a TypeError and an IndexError so I'm going to capture all exceptions raised
                pass
            else:
                v["floorplan_url"] = floorplan_url
            # Scrape the pictures of the house ### commented this out for now, as I'm not using the image data
            #house_image_urls = []
            #for image in json_dict["soldPropertyData"]["property"]["images"]:
            #	house_image_urls.append(image['url'])
            #v["house_image_urls"] = house_image_urls
            # location of the house
            try:
                latitude = json_dict["soldPropertyData"]["property"]["location"]["latitude"]
                longitude = json_dict["soldPropertyData"]["property"]["location"]["longitude"]
            except TypeError:
                pass
            else:
                v["latitude"] = latitude
                v["longitude"] = longitude
            # proximity to the stations 
            station_names, distances = [], []
            try:
                for station in json_dict["soldPropertyData"]["property"]["nearestStations"]:
                    station_names.append(station["name"].replace("'", ""))
                    distances.append(station["distance"])
            except TypeError:
                station_proximities = {}
            else:
                station_proximities = {station:distance for station, distance in zip(station_names, distances)}
            v["station_proximities"] = station_proximities
            # size of the property
            units, sizes = [], []
            try:
                for i in json_dict["soldPropertyData"]["property"]["sizings"]:
                    units.append(i["unit"])
                    sizes.append(i["maximumSize"])
            except TypeError:
                dimensions = ""
            else:
                dimensions = {unit:size for unit, size in zip(units, sizes)}
            #for label, size in dimensions.items():
             #   if label not in m_values and label not in f_values:
              #      del dimensions[label]

            

            v["property_size"] = dimensions
            
            # JUST ABOVE HERE IS WHERE I COULD MAKE IT SO ONLY SQ FT IS OBTAINED - Still needs to be finished off

            # number of bedrooms and bathrooms
            try:
                bedrooms = json_dict["soldPropertyData"]["property"]["bedrooms"]
            except TypeError:
                bedrooms = ''
            try:
                bathrooms = json_dict["soldPropertyData"]["property"]["bathrooms"]
            except TypeError:
                bathrooms = ''
                
            v["bedrooms"] = bedrooms
            v["bathrooms"] = bathrooms

            # Whether the property is a new build or not
            try:
                new_build = json_dict["soldPropertyData"]["transactions"][0]["newBuild"]
            except TypeError:
                pass
            else:
                v["new_build"] = new_build
                
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(scrape_data, local_area_urls[:])
    
with open("houses600.json", "w") as fbj:
    json.dump(houses, fbj)

In [ ]:
with open("houses600.json", "w") as fbj:
    json.dump(houses, fbj)

In [ ]:
with open("houses600.json", "w") as fbj:
    json.dump(houses, fbj)

In [ ]:
# change price from a string into an int
def price_int(x):
    """Return an integer from a price string"""
    a = x[1:]
    b = a.replace(",", "")
    b = int(b)
    return b

# run the above function for all of the stored prices
for features in houses.values():
    y = features['price']
    z = price_int(y)
    features['price'] = z

In [ ]:
def size_floorplan_2(floorplan):
    """Return the size data (in sq ft) from the floorplan"""
    reader = easyocr.Reader(['en']) 
    result = reader.readtext(floorplan)
    
    scraped_text = ''
    for i in result:
        scraped_text += i[1]
        scraped_text += ","
    text_list = scraped_text.split(",")
    
    m_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square meters|square metres|square meter|square metre|square m|square mt|sqmt|sq mt|sq.mt|sq. mt|sqm|sq m|sq.m|sq. m|sq: m|sq:m|sq :m|sq : m|sq :m|meters2|metres2|meter2|metre2|mt2|m2)", re.IGNORECASE)
    f_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square feet|square ft|square f|sqft|sq ft|sq.ft|sq. ft|sqf|sq f|sq.f|sq. f|sq: f|sq:f|sq :f|sq : f|sq :f|feet2|ft2|f2)", re.IGNORECASE)
    house_size = []
    for x in text_list:
        for match in re.finditer(m_patterns, x):
            #print('%s : %s' % (match.groups()))
            house_size.append(list(match.groups()))
        for match in re.finditer(f_patterns, x):
            #print('%s : %s' % (match.groups())) 
            house_size.append(list(match.groups()))
    for a in house_size:
        try:
            a[0] = int(a[0])
        except ValueError:
            a[0] = float(a[0])
            
    # then here you could find the
    house_size = dict((house_size))
    #print(house_size)
    house_size_swap = {}
    # code to swap keys and values
    for k, v in house_size.items():
        house_size_swap[v] = k

    house_size = house_size_swap
    
    # Code to take the largest returned value from the floorplan numbers
    amount = max(house_size.values())
    for i in house_size.keys():
        if house_size[i] == amount:
            size_label = i.lower()
    
    # Remove punctuation from the size labels       
    if "," in size_label:
        size_label = size_label.replace(",", "")
    if ":" in size_label:
        size_label = size_label.replace(":", "")
    if "." in size_label:
        size_label = size_label.replace(".", "")
    
    largest_size = {size_label:amount}
    
    
            
    # need to add values with colons, :
    # alternatively, you could just remove the colon. This would also nehate this issue

    metre_values = ["square meters", "square metres", "square meter", "square metre", "square m", "square mt", "sqmt", "sq mt", "sq mt", "sq mt", "sqm", "sq m", "sq m", "sq m", "meters2", "metres2", "meter2", "metre2", "mt2", "m2"]
    feet_values = ["square feet","square ft","square f","sqft","sq ft","sq  ft","sq. ft","sqf","sq f","sq f","sq f","feet2", "ft2","f2"]


    if size_label in metre_values:
        # convert the size into square ft if sq m is obtained
        largest_size["sq ft"] = amount*10.764
        del largest_size[size_label]
    elif size_label in feet_values:
        largest_size["sq ft"] = amount
    

        
    return largest_size["sq ft"]

In [ ]:
# function that returns the sq foot value from the dictionaries that are scraped from the sold houses
def sq_foot_return(x):
    values_m = ["square meters", "sq meters", "square metres", "sq metres", "square meter", "sq meter", "square metre", "sq metre","square m", "square mt", "sqmt", "sq mt", "sq.mt", "sq. mt", "sqm", "sq m", "sq.m", "sq. m", "sq: m", "sq:m", "sq :m", "sq : m", "sq :m", "meters2", "metres2", "meter2", "metre2", "mt2", "m2"]
    values_f = ["square feet", "square ft", "square f", "sqft", "sq ft", "sq.ft", "sq. ft", "sqf", "sq f", "sq.f", "sq. f", "sq: f", "sq:f", "sq :f", "sq : f", "sq :f", "feet2", "ft2", "f2"]
    for k, v in x.items():
        if k.strip() in values_f:
            return x[k]
        if k.strip() in values_m:
            return x[k]*10.764
    return "Na"

In [ ]:
# here is where the code should go to create this outstanding schools list
outstanding_schools = [[51.4974948, -0.1356583],
 [51.5202607, -0.0293396],
 [51.4935082, -0.1178424],
 [51.6569225, -0.1949252],
 [51.5202607, -0.0293396],
 [51.5345448, -0.2043853],
 [51.5202607, -0.0293396],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.6569225, -0.1949252],
 [51.5906113, -0.1109709],
 [51.5202607, -0.0293396],
 [51.538621, -0.1028346],
 [51.5202607, -0.0293396],
 [51.5672808, -0.2710568],
 [51.6569225, -0.1949252],
 [51.5132537, -0.3043136],
 [51.5255162, 0.0352163],
 [51.5202607, -0.0293396],
 [51.4609218, -0.373149],
 [51.6569225, -0.1949252],
 [51.4990805, -0.1938253],
 [51.5255162, 0.0352163],
 [51.5255162, 0.0352163],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.5202607, -0.0293396],
 [51.5436387, -0.0553621],
 [51.4935082, -0.1178424],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.5886383, -0.0117625],
 [51.5390261, -0.1425516],
 [51.5132537, -0.3043136],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.5886121, 0.0823982],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.5886383, -0.0117625],
 [51.5132537, -0.3043136],
 [51.5906113, -0.1109709],
 [51.458373, -0.1891356],
 [51.5202607, -0.0293396],
 [51.4990805, -0.1938253],
 [51.5202607, -0.0293396],
 [51.5672808, -0.2710568],
 [51.5886383, -0.0117625],
 [51.4990156, -0.22915],
 [51.5132537, -0.3043136],
 [51.5202607, -0.0293396],
 [51.4609218, -0.373149],
 [51.4974948, -0.1356583],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.4933675, 0.0098214],
 [51.5886121, 0.0823982],
 [51.5886121, 0.0823982],
 [51.502781, -0.087738],
 [51.4935082, -0.1178424],
 [51.5540666, 0.134017],
 [51.4935082, -0.1178424],
 [51.5886121, 0.0823982],
 [51.5886383, -0.0117625],
 [51.4611509, -0.0073177],
 [51.5390261, -0.1425516],
 [51.4609218, -0.373149],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5132537, -0.3043136],
 [51.4990805, -0.1938253],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.5886121, 0.0823982],
 [51.5672808, -0.2710568],
 [51.5436387, -0.0553621],
 [51.5886121, 0.0823982],
 [51.5202607, -0.0293396],
 [51.5390261, -0.1425516],
 [51.502781, -0.087738],
 [51.4609218, -0.373149],
 [51.5436387, -0.0553621],
 [51.4974948, -0.1356583],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.5123443, -0.0909852],
 [51.538621, -0.1028346],
 [51.5672808, -0.2710568],
 [51.5436387, -0.0553621],
 [51.5255162, 0.0352163],
 [51.5886383, -0.0117625],
 [51.538621, -0.1028346],
 [51.4935082, -0.1178424],
 [51.4990156, -0.22915],
 [51.538621, -0.1028346],
 [51.5906113, -0.1109709],
 [51.5906113, -0.1109709],
 [51.5906113, -0.1109709],
 [51.502781, -0.087738],
 [51.4935082, -0.1178424],
 [51.5202607, -0.0293396],
 [51.502781, -0.087738],
 [51.5202607, -0.0293396],
 [51.5540666, 0.134017],
 [51.4990156, -0.22915],
 [51.4609218, -0.373149],
 [51.4935082, -0.1178424],
 [51.5132537, -0.3043136],
 [51.6569225, -0.1949252],
 [51.4611509, -0.0073177],
 [51.4935082, -0.1178424],
 [51.538621, -0.1028346],
 [51.5202607, -0.0293396],
 [51.4933675, 0.0098214],
 [51.406025, 0.013156],
 [51.5886383, -0.0117625],
 [51.458373, -0.1891356],
 [51.5886383, -0.0117625],
 [51.502781, -0.087738],
 [51.4990805, -0.1938253],
 [51.4097742, -0.2108084],
 [51.4933675, 0.0098214],
 [51.6522994, -0.0807119],
 [51.5672808, -0.2710568],
 [51.502781, -0.087738],
 [51.4974948, -0.1356583],
 [51.580559, -0.341995],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.6569225, -0.1949252],
 [51.4611509, -0.0073177],
 [51.6522994, -0.0807119],
 [51.5132537, -0.3043136],
 [51.6522994, -0.0807119],
 [51.5132537, -0.3043136],
 [51.458373, -0.1891356],
 [51.4097742, -0.2108084],
 [51.6522994, -0.0807119],
 [51.5436387, -0.0553621],
 [51.4611509, -0.0073177],
 [51.502781, -0.087738],
 [51.6522994, -0.0807119],
 [51.4974948, -0.1356583],
 [51.458373, -0.1891356],
 [51.502781, -0.087738],
 [51.458373, -0.1891356],
 [51.5540666, 0.134017],
 [51.461311, -0.303742],
 [51.502781, -0.087738],
 [51.5672808, -0.2710568],
 [51.461311, -0.303742],
 [51.5906113, -0.1109709],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.458373, -0.1891356],
 [51.5886383, -0.0117625],
 [51.5886121, 0.0823982],
 [51.5672808, -0.2710568],
 [51.4609218, -0.373149],
 [51.4935082, -0.1178424],
 [51.577924, 0.2120829],
 [51.4990805, -0.1938253],
 [51.5886383, -0.0117625],
 [51.461311, -0.303742],
 [51.4935082, -0.1178424],
 [51.4990156, -0.22915],
 [51.5255162, 0.0352163],
 [51.502781, -0.087738],
 [51.5255162, 0.0352163],
 [51.5132537, -0.3043136],
 [51.5255162, 0.0352163],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.538621, -0.1028346],
 [51.3769529, -0.0956895],
 [51.6569225, -0.1949252],
 [51.580559, -0.341995],
 [51.5436387, -0.0553621],
 [51.5255162, 0.0352163],
 [51.4990805, -0.1938253],
 [51.439933, 0.154327],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.3769529, -0.0956895],
 [51.5436387, -0.0553621],
 [51.439933, 0.154327],
 [51.6569225, -0.1949252],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.4097742, -0.2108084],
 [51.6522994, -0.0807119],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.5886121, 0.0823982],
 [51.41233, -0.300689],
 [51.5436387, -0.0553621],
 [51.406025, 0.013156],
 [51.3769529, -0.0956895],
 [51.502781, -0.087738],
 [51.580559, -0.341995],
 [51.439933, 0.154327],
 [51.5906113, -0.1109709],
 [51.5351832, -0.4481378],
 [51.5255162, 0.0352163],
 [51.4974948, -0.1356583],
 [51.4097742, -0.2108084],
 [51.5436387, -0.0553621],
 [51.5202607, -0.0293396],
 [51.5390261, -0.1425516],
 [51.6569225, -0.1949252],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.4935082, -0.1178424],
 [51.5540666, 0.134017],
 [51.4974948, -0.1356583],
 [51.4933675, 0.0098214],
 [51.5540666, 0.134017],
 [51.3769529, -0.0956895],
 [51.5351832, -0.4481378],
 [51.4990156, -0.22915],
 [51.4935082, -0.1178424],
 [51.4611509, -0.0073177],
 [51.4611509, -0.0073177],
 [51.5886121, 0.0823982],
 [51.5906113, -0.1109709],
 [51.6522994, -0.0807119],
 [51.6522994, -0.0807119],
 [51.3769529, -0.0956895],
 [51.458373, -0.1891356],
 [51.4933675, 0.0098214],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.4097742, -0.2108084],
 [51.4935082, -0.1178424],
 [51.4611509, -0.0073177],
 [51.502781, -0.087738],
 [51.577924, 0.2120829],
 [51.406025, 0.013156],
 [51.5906113, -0.1109709],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.439933, 0.154327],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.5132537, -0.3043136],
 [51.4611509, -0.0073177],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.5202607, -0.0293396],
 [51.4935082, -0.1178424],
 [51.4611509, -0.0073177],
 [51.4990805, -0.1938253],
 [51.6522994, -0.0807119],
 [51.5886121, 0.0823982],
 [51.439933, 0.154327],
 [51.4935082, -0.1178424],
 [51.5906113, -0.1109709],
 [51.41233, -0.300689],
 [51.5906113, -0.1109709],
 [51.4611509, -0.0073177],
 [51.4611509, -0.0073177],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.4609218, -0.373149],
 [51.4611509, -0.0073177],
 [51.5906113, -0.1109709],
 [51.580559, -0.341995],
 [51.3769529, -0.0956895],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.5886121, 0.0823982],
 [51.461311, -0.303742],
 [51.4935082, -0.1178424],
 [51.5132537, -0.3043136],
 [51.6569225, -0.1949252],
 [51.4097742, -0.2108084],
 [51.5906113, -0.1109709],
 [51.6569225, -0.1949252],
 [51.5886383, -0.0117625],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.4609218, -0.373149],
 [51.577924, 0.2120829],
 [51.5351832, -0.4481378],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.5906113, -0.1109709],
 [51.5886121, 0.0823982],
 [51.5672808, -0.2710568],
 [51.439933, 0.154327],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.5886121, 0.0823982],
 [51.439933, 0.154327],
 [51.5351832, -0.4481378],
 [51.4609218, -0.373149],
 [51.5540666, 0.134017],
 [51.6569225, -0.1949252],
 [51.3769529, -0.0956895],
 [51.439933, 0.154327],
 [51.6569225, -0.1949252],
 [51.4609218, -0.373149],
 [51.4990805, -0.1938253],
 [51.3769529, -0.0956895],
 [51.580559, -0.341995],
 [51.4974948, -0.1356583],
 [51.3769529, -0.0956895],
 [51.4990156, -0.22915],
 [51.5351832, -0.4481378],
 [51.5202607, -0.0293396],
 [51.458373, -0.1891356],
 [51.461311, -0.303742],
 [51.5906113, -0.1109709],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.5132537, -0.3043136],
 [51.4609218, -0.373149],
 [51.5436387, -0.0553621],
 [51.5390261, -0.1425516],
 [51.5436387, -0.0553621],
 [51.406025, 0.013156],
 [51.502781, -0.087738],
 [51.458373, -0.1891356],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.461311, -0.303742],
 [51.5436387, -0.0553621],
 [51.5351832, -0.4481378],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.577924, 0.2120829],
 [51.3769529, -0.0956895],
 [51.4974948, -0.1356583],
 [51.5390261, -0.1425516],
 [51.458373, -0.1891356],
 [51.4974948, -0.1356583],
 [51.458373, -0.1891356],
 [51.5255162, 0.0352163],
 [51.4974948, -0.1356583],
 [51.6522994, -0.0807119],
 [51.4990156, -0.22915],
 [51.4990805, -0.1938253],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5202607, -0.0293396],
 [51.5886383, -0.0117625],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.5202607, -0.0293396],
 [51.5540666, 0.134017],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.4097742, -0.2108084],
 [51.5390261, -0.1425516],
 [51.4990156, -0.22915],
 [51.458373, -0.1891356],
 [51.5123443, -0.0909852],
 [51.461311, -0.303742],
 [51.458373, -0.1891356],
 [51.4097742, -0.2108084],
 [51.538621, -0.1028346],
 [51.5886121, 0.0823982],
 [51.4974948, -0.1356583],
 [51.458373, -0.1891356],
 [51.6569225, -0.1949252],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.6569225, -0.1949252],
 [51.4935082, -0.1178424],
 [51.580559, -0.341995],
 [51.4935082, -0.1178424],
 [51.538621, -0.1028346],
 [51.6522994, -0.0807119],
 [51.5672808, -0.2710568],
 [51.5255162, 0.0352163],
 [51.4990805, -0.1938253],
 [51.4974948, -0.1356583],
 [51.4990156, -0.22915],
 [51.5672808, -0.2710568],
 [51.6522994, -0.0807119],
 [51.538621, -0.1028346],
 [51.5906113, -0.1109709],
 [51.6522994, -0.0807119],
 [51.4974948, -0.1356583],
 [51.5132537, -0.3043136],
 [51.5202607, -0.0293396],
 [51.5132537, -0.3043136],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5351832, -0.4481378],
 [51.5390261, -0.1425516],
 [51.580559, -0.341995],
 [51.458373, -0.1891356],
 [51.538621, -0.1028346],
 [51.4935082, -0.1178424],
 [51.580559, -0.341995],
 [51.5351832, -0.4481378],
 [51.502781, -0.087738],
 [51.4990156, -0.22915],
 [51.5886121, 0.0823982],
 [51.6569225, -0.1949252],
 [51.4990805, -0.1938253],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.5132537, -0.3043136],
 [51.3769529, -0.0956895],
 [51.458373, -0.1891356],
 [51.5351832, -0.4481378],
 [51.4990805, -0.1938253],
 [51.502781, -0.087738],
 [51.458373, -0.1891356],
 [51.6522994, -0.0807119],
 [51.6569225, -0.1949252],
 [51.5672808, -0.2710568],
 [51.538621, -0.1028346],
 [51.5202607, -0.0293396],
 [51.4990805, -0.1938253],
 [51.6522994, -0.0807119],
 [51.4974948, -0.1356583],
 [51.4611509, -0.0073177],
 [51.580559, -0.341995],
 [51.577924, 0.2120829],
 [51.3769529, -0.0956895],
 [51.6569225, -0.1949252],
 [51.458373, -0.1891356],
 [51.502781, -0.087738],
 [51.41233, -0.300689],
 [51.5540666, 0.134017],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.5351832, -0.4481378],
 [51.6522994, -0.0807119],
 [51.5390261, -0.1425516],
 [51.5132537, -0.3043136],
 [51.502781, -0.087738],
 [51.5132537, -0.3043136],
 [51.5351832, -0.4481378],
 [51.5202607, -0.0293396],
 [51.4990805, -0.1938253],
 [51.5672808, -0.2710568],
 [51.3769529, -0.0956895],
 [51.4990805, -0.1938253],
 [51.4990156, -0.22915],
 [51.4974948, -0.1356583],
 [51.5886121, 0.0823982],
 [51.461311, -0.303742],
 [51.5351832, -0.4481378],
 [51.5390261, -0.1425516],
 [51.5255162, 0.0352163],
 [51.538621, -0.1028346],
 [51.4097742, -0.2108084],
 [51.5886121, 0.0823982],
 [51.3769529, -0.0956895],
 [51.5390261, -0.1425516],
 [51.577924, 0.2120829],
 [51.5886383, -0.0117625],
 [51.406025, 0.013156],
 [51.4990805, -0.1938253],
 [51.3769529, -0.0956895],
 [51.6522994, -0.0807119],
 [51.4935082, -0.1178424],
 [51.461311, -0.303742],
 [51.580559, -0.341995],
 [51.3769529, -0.0956895],
 [51.580559, -0.341995],
 [51.406025, 0.013156],
 [51.6522994, -0.0807119],
 [51.4990156, -0.22915],
 [51.4611509, -0.0073177],
 [51.4933675, 0.0098214],
 [51.4974948, -0.1356583],
 [51.6522994, -0.0807119],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.5886383, -0.0117625],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.4609218, -0.373149],
 [51.406025, 0.013156],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.5672808, -0.2710568],
 [51.5672808, -0.2710568],
 [51.4990805, -0.1938253],
 [51.5202607, -0.0293396],
 [51.5202607, -0.0293396],
 [51.580559, -0.341995],
 [51.5436387, -0.0553621],
 [51.538621, -0.1028346],
 [51.458373, -0.1891356],
 [51.458373, -0.1891356],
 [51.580559, -0.341995],
 [51.5390261, -0.1425516],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.5906113, -0.1109709],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.458373, -0.1891356],
 [51.461311, -0.303742],
 [51.5886383, -0.0117625],
 [51.5390261, -0.1425516],
 [51.458373, -0.1891356],
 [51.580559, -0.341995],
 [51.5255162, 0.0352163],
 [51.5906113, -0.1109709],
 [51.4990156, -0.22915],
 [51.4990805, -0.1938253],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.4611509, -0.0073177],
 [51.3769529, -0.0956895],
 [51.4933675, 0.0098214],
 [51.577924, 0.2120829],
 [51.6522994, -0.0807119],
 [51.458373, -0.1891356],
 [51.4933675, 0.0098214],
 [51.4611509, -0.0073177],
 [51.4933675, 0.0098214],
 [51.4609218, -0.373149],
 [51.406025, 0.013156],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.6522994, -0.0807119],
 [51.4933675, 0.0098214],
 [51.4974948, -0.1356583],
 [51.5906113, -0.1109709],
 [51.5351832, -0.4481378],
 [51.5436387, -0.0553621],
 [51.4990805, -0.1938253],
 [51.6569225, -0.1949252],
 [51.461311, -0.303742],
 [51.6569225, -0.1949252],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.4990805, -0.1938253],
 [51.5886121, 0.0823982],
 [51.5351832, -0.4481378],
 [51.6522994, -0.0807119],
 [51.5255162, 0.0352163],
 [51.461311, -0.303742],
 [51.4974948, -0.1356583],
 [51.502781, -0.087738],
 [51.5436387, -0.0553621],
 [51.5906113, -0.1109709],
 [51.580559, -0.341995],
 [51.5202607, -0.0293396],
 [51.5886121, 0.0823982],
 [51.439933, 0.154327],
 [51.5436387, -0.0553621],
 [51.458373, -0.1891356],
 [51.5255162, 0.0352163],
 [51.6569225, -0.1949252],
 [51.4933675, 0.0098214],
 [51.580559, -0.341995],
 [51.4974948, -0.1356583],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.4974948, -0.1356583],
 [51.502781, -0.087738],
 [51.5132537, -0.3043136],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.580559, -0.341995],
 [51.439933, 0.154327],
 [51.4611509, -0.0073177],
 [51.4933675, 0.0098214],
 [51.458373, -0.1891356],
 [51.4097742, -0.2108084],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.580559, -0.341995],
 [51.4935082, -0.1178424],
 [51.461311, -0.303742],
 [51.439933, 0.154327],
 [51.3769529, -0.0956895],
 [51.4935082, -0.1178424],
 [51.458373, -0.1891356],
 [51.439933, 0.154327],
 [51.580559, -0.341995],
 [51.4933675, 0.0098214],
 [51.461311, -0.303742],
 [51.3769529, -0.0956895],
 [51.577924, 0.2120829],
 [51.5436387, -0.0553621],
 [51.406025, 0.013156],
 [51.5132537, -0.3043136],
 [51.4974948, -0.1356583],
 [51.5886383, -0.0117625],
 [51.4990156, -0.22915],
 [51.406025, 0.013156],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.3769529, -0.0956895],
 [51.538621, -0.1028346],
 [51.5436387, -0.0553621],
 [51.538621, -0.1028346],
 [51.439933, 0.154327],
 [51.458373, -0.1891356],
 [51.3769529, -0.0956895],
 [51.5672808, -0.2710568],
 [51.4935082, -0.1178424],
 [51.41233, -0.300689],
 [51.577924, 0.2120829],
 [51.6569225, -0.1949252],
 [51.5202607, -0.0293396],
 [51.6569225, -0.1949252],
 [51.41233, -0.300689],
 [51.3769529, -0.0956895],
 [51.458373, -0.1891356],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.4609218, -0.373149],
 [51.5886121, 0.0823982],
 [51.458373, -0.1891356],
 [51.5132537, -0.3043136],
 [51.458373, -0.1891356],
 [51.4611509, -0.0073177],
 [51.461311, -0.303742],
 [51.4933675, 0.0098214],
 [51.4097742, -0.2108084],
 [51.5436387, -0.0553621],
 [51.577924, 0.2120829],
 [51.4990156, -0.22915],
 [51.406025, 0.013156],
 [51.580559, -0.341995],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.4990156, -0.22915],
 [51.458373, -0.1891356],
 [51.5123443, -0.0909852],
 [51.4990156, -0.22915],
 [51.5906113, -0.1109709],
 [51.3769529, -0.0956895],
 [51.580559, -0.341995],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.3769529, -0.0956895],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.5886121, 0.0823982],
 [51.4609218, -0.373149],
 [51.3769529, -0.0956895],
 [51.538621, -0.1028346],
 [51.6569225, -0.1949252],
 [51.5351832, -0.4481378],
 [51.6569225, -0.1949252],
 [51.4933675, 0.0098214],
 [51.458373, -0.1891356],
 [51.4935082, -0.1178424],
 [51.4974948, -0.1356583],
 [51.4609218, -0.373149],
 [51.6569225, -0.1949252],
 [51.406025, 0.013156],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.461311, -0.303742],
 [51.461311, -0.303742],
 [51.5351832, -0.4481378],
 [51.5540666, 0.134017],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.458373, -0.1891356],
 [51.5672808, -0.2710568],
 [51.458373, -0.1891356],
 [51.4974948, -0.1356583],
 [51.4974948, -0.1356583],
 [51.5906113, -0.1109709],
 [51.5906113, -0.1109709],
 [51.5540666, 0.134017],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5390261, -0.1425516],
 [51.4609218, -0.373149],
 [51.4990156, -0.22915],
 [51.6522994, -0.0807119],
 [51.5672808, -0.2710568],
 [51.5436387, -0.0553621],
 [51.502781, -0.087738],
 [51.461311, -0.303742],
 [51.5436387, -0.0553621],
 [51.5351832, -0.4481378],
 [51.5886121, 0.0823982],
 [51.580559, -0.341995],
 [51.4097742, -0.2108084],
 [51.5886383, -0.0117625],
 [51.4611509, -0.0073177],
 [51.577924, 0.2120829],
 [51.5132537, -0.3043136],
 [51.5390261, -0.1425516],
 [51.5132537, -0.3043136],
 [51.502781, -0.087738],
 [51.538621, -0.1028346],
 [51.4097742, -0.2108084],
 [51.5132537, -0.3043136],
 [51.461311, -0.303742],
 [51.5886383, -0.0117625],
 [51.6569225, -0.1949252],
 [51.41233, -0.300689],
 [51.577924, 0.2120829],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.458373, -0.1891356],
 [51.5202607, -0.0293396],
 [51.502781, -0.087738],
 [51.577924, 0.2120829],
 [51.5906113, -0.1109709],
 [51.4609218, -0.373149],
 [51.4609218, -0.373149],
 [51.5540666, 0.134017],
 [51.577924, 0.2120829],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.4933675, 0.0098214],
 [51.5886383, -0.0117625],
 [51.406025, 0.013156],
 [51.439933, 0.154327],
 [51.3769529, -0.0956895],
 [51.5202607, -0.0293396],
 [51.461311, -0.303742],
 [51.4974948, -0.1356583],
 [51.461311, -0.303742],
 [51.5886383, -0.0117625],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.458373, -0.1891356],
 [51.5886121, 0.0823982],
 [51.4097742, -0.2108084],
 [51.458373, -0.1891356],
 [51.458373, -0.1891356],
 [51.4611509, -0.0073177],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.4935082, -0.1178424],
 [51.406025, 0.013156],
 [51.458373, -0.1891356],
 [51.4990805, -0.1938253],
 [51.6569225, -0.1949252],
 [51.6569225, -0.1949252],
 [51.5672808, -0.2710568],
 [51.5132537, -0.3043136],
 [51.5886383, -0.0117625],
 [51.458373, -0.1891356],
 [51.3769529, -0.0956895],
 [51.5390261, -0.1425516],
 [51.4990156, -0.22915],
 [51.3769529, -0.0956895],
 [51.5886121, 0.0823982],
 [51.406025, 0.013156],
 [51.5202607, -0.0293396],
 [51.461311, -0.303742],
 [51.580559, -0.341995],
 [51.6569225, -0.1949252],
 [51.5390261, -0.1425516],
 [51.406025, 0.013156],
 [51.5886121, 0.0823982],
 [51.538621, -0.1028346],
 [51.461311, -0.303742],
 [51.461311, -0.303742],
 [51.5886121, 0.0823982],
 [51.6569225, -0.1949252],
 [51.580559, -0.341995],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.461311, -0.303742],
 [51.580559, -0.341995],
 [51.4990156, -0.22915],
 [51.5351832, -0.4481378],
 [51.5132537, -0.3043136],
 [51.5255162, 0.0352163],
 [51.5255162, 0.0352163],
 [51.538621, -0.1028346],
 [51.4609218, -0.373149],
 [51.5390261, -0.1425516],
 [51.5202607, -0.0293396],
 [51.580559, -0.341995],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.4990156, -0.22915],
 [51.4609218, -0.373149],
 [51.4935082, -0.1178424],
 [51.5390261, -0.1425516],
 [51.461311, -0.303742],
 [51.5132537, -0.3043136],
 [51.461311, -0.303742],
 [51.3769529, -0.0956895],
 [51.5540666, 0.134017],
 [51.4933675, 0.0098214],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.5906113, -0.1109709],
 [51.5255162, 0.0352163],
 [51.580559, -0.341995],
 [51.5886121, 0.0823982],
 [51.458373, -0.1891356],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.6569225, -0.1949252],
 [51.5672808, -0.2710568],
 [51.5672808, -0.2710568],
 [51.538621, -0.1028346],
 [51.5540666, 0.134017],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.5132537, -0.3043136],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.4933675, 0.0098214],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.461311, -0.303742],
 [51.538621, -0.1028346],
 [51.4990156, -0.22915],
 [51.4935082, -0.1178424],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.41233, -0.300689],
 [51.406025, 0.013156],
 [51.5351832, -0.4481378],
 [51.5132537, -0.3043136],
 [51.538621, -0.1028346],
 [51.41233, -0.300689],
 [51.538621, -0.1028346],
 [51.4990805, -0.1938253],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.4974948, -0.1356583],
 [51.4935082, -0.1178424],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.5255162, 0.0352163],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.4974948, -0.1356583],
 [51.538621, -0.1028346],
 [51.538621, -0.1028346],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.5202607, -0.0293396],
 [51.458373, -0.1891356],
 [51.4935082, -0.1178424],
 [51.4933675, 0.0098214],
 [51.3769529, -0.0956895],
 [51.4990156, -0.22915],
 [51.5390261, -0.1425516],
 [51.4097742, -0.2108084],
 [51.4990805, -0.1938253],
 [51.4097742, -0.2108084],
 [51.5202607, -0.0293396],
 [51.580559, -0.341995],
 [51.577924, 0.2120829],
 [51.4974948, -0.1356583],
 [51.406025, 0.013156],
 [51.5886121, 0.0823982],
 [51.5436387, -0.0553621],
 [51.5351832, -0.4481378],
 [51.580559, -0.341995],
 [51.4974948, -0.1356583],
 [51.502781, -0.087738],
 [51.4609218, -0.373149],
 [51.5255162, 0.0352163],
 [51.580559, -0.341995],
 [51.5886383, -0.0117625],
 [51.4990805, -0.1938253],
 [51.502781, -0.087738],
 [51.3769529, -0.0956895],
 [51.577924, 0.2120829],
 [51.577924, 0.2120829],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.538621, -0.1028346],
 [51.4609218, -0.373149],
 [51.406025, 0.013156],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.4990805, -0.1938253],
 [51.4935082, -0.1178424],
 [51.406025, 0.013156],
 [51.4933675, 0.0098214],
 [51.458373, -0.1891356],
 [51.6569225, -0.1949252],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.502781, -0.087738],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.5886121, 0.0823982],
 [51.577924, 0.2120829],
 [51.4611509, -0.0073177],
 [51.4611509, -0.0073177]]

In [ ]:
def nearest_outstanding_school(lat, lon):
    """Return the KM distance to the nearest outstanding school"""
    coords_1 = (lat, lon)
    lower_boundary, upper_boundary = -0.05, 0.05
    school_distances = []
    for i in outstanding_schools:
        if (i[0] - lat > upper_boundary) or i[0] - lat < lower_boundary:
            continue
        elif (i[1] - lon > upper_boundary) or i[1] - lon < lower_boundary:
            continue
        else:
            coords_2 = (i[0], i[1])
            school_distances.append(geopy.distance.geodesic(coords_1, coords_2).km)
            
    try:
        return min(school_distances)
    except ValueError:
        return "Na"

In [ ]:
# get all of the latitude and longitudes for all of the houses in the data set, and add this to the data.
nomi = pgeocode.Nominatim('GB')
for x, y in houses.items():
    if 'latitude' in y.keys() and 'longitude' in y.keys():
        continue
    else:
        postcode = x[-8:]
        if postcode[0] == " ":
            postcode = x[-7:]
        postcode_split = postcode.split(" ")
        if len(postcode_split) == 2:
            postcode = postcode_split[0]+" "+postcode_split[1]
        elif len(postcode_split) == 3:
            postcode = postcode_split[1]+" "+postcode_split[2]
        lat = float(nomi.query_postal_code(postcode).latitude)
        lon = float(nomi.query_postal_code(postcode).longitude)
        
        y["latitude"] = lat
        y["longitude"] = lon

In [ ]:
# Updated code for property size 
for y in houses.values():
    if 'property_size' in y.keys():
        if type(y['property_size']) == int or type(y['property_size']) == float:
            continue
        elif type(y['property_size']) == dict and y['property_size'] != {}:
            value = sq_foot_return(y['property_size'])
            y['property_size'] = value
            continue
        elif 'floorplan_url' in y.keys():
            try:
                ocr_size = size_floorplan_2(y['floorplan_url'])
                y['property_size'] = ocr_size
            except Exception:
                y['property_size'] = 'Na'

for y in houses.values():
    if 'latitude' in y.keys() and 'longitude' in y.keys():
        distance_km = nearest_outstanding_school(y['latitude'], y['longitude'])
        y['nearest_outstanding_school'] = distance_km
                
            

In [ ]:
for y in houses.values():
    print(y)
    print()
    try:
        print(y['floorplan_url'])
        print()
    except KeyError:
        print()

In [ ]:
# for y in houses.values():
#     if 'property_size' in y.keys():
#         try:
#             if len(y["property_size"]) >= 1 and type(y["property_size"]) == dict:
#                 value = sq_foot_return(y['property_size'])
#                 y['property_size'] = value
#         except TypeError:
#             pass
#         if 'latitude' in y.keys() and 'longitude' in y.keys():
#             distance_km = nearest_outstanding_school(y['latitude'], y['longitude'])
#             y['nearest_outstanding_school'] = distance_km

In [ ]:
# # loop through all of the houses, if they don't have a property size but do have a floorplan, apply the OCR, and then return 
# # a value (note a nonetype may be returned so you'll have to deal with that). 

# for y in houses.values():
#     if 'property_size' in y.keys():
#         if type(y['property_size']) == int or type(y['property_size']) == float:
#             continue
#         elif (y['property_size'] == {} or type(y['property_size']) == str) and ('floorplan_url' in y.keys()):
#             try:
#                 ocr_size = size_floorplan_2(y['floorplan_url'])
#             except Exception:
#                 y['property_size'] = 'Na'
#                 print("Na")
#             else:
#                 if ocr_size == None or ocr_size == '':
#                     y['property_size'] = 'Na'
#                     print("Na")
#                 else:
#                     y['property_size'] = ocr_size
#                     print(ocr_size)
                

In [ ]:
houses1

In [ ]:
# This is for the first 10 URL's 
with open("houses1.json") as f:
    contents = f.read()
len(contents)
    #houses1all = json.loads(contents)
    
houses1A = json.loads(contents[:3999734]+'}')
houses1B = json.loads('{'+contents[3999736:])

with open("houses1A.json", "w") as fb:
    json.dump(houses1A, fb)
    
with open("houses1B.json", "w") as fb:
    json.dump(houses1B, fb)

In [ ]:
# First 100 URL's 
with open("slurm-100A.out") as f:
    contents = f.read()
contents


In [ ]:
# Format the string so JSON can convert it into a python dict
with open("slurm-houses1A.out ", encoding="utf-8") as f:
    contents = f.read()
    # Remove the progress bars produced by the OCR
    start = contents.find("{")
    data1A = contents[start:-1]
    data1A = data1A.replace("False", "false")
    data1A = data1A.replace("True", "true")
    data1A = data1A.replace("None", "null")
    data1A = data1A.replace("nan", "null")
    data1A = data1A.replace("'", '"')
    
with open("slurm-houses1B.out ", encoding="utf-8") as f:
    contents1 = f.read()
    # Remove the progress bars produced by the OCR
    start = contents1.find("{")
    data2A = contents1[start:-1]
    data2A = data2A.replace("False", "false")
    data2A = data2A.replace("True", "true")
    data2A = data2A.replace("None", "null")
    data2A = data2A.replace("nan", "null")
    data2A = data2A.replace("'", '"')

data1AB = data1A[:-1]+', '+data2A[1:]


In [ ]:
houses1AB = json.loads(data1AB)

In [ ]:
type(houses1AB)

In [ ]:
len(houses1AB)


In [ ]:
# This is where the NLP should be done # 

In [ ]:
## LET'S LEAVE THIS FOR NOW!!!
##############################
# Code for finding the average price of houses in the same postcode area with the same number of bedrooms of each property
# This column will be used in the first regression 

# for i, v in houses1.items():
#     local_similar_prices = []
#     # need to store the postcode, each house has a postcode so this will be easier
#     postcode = i[-8:]
#     if postcode[0] == " ":
#         postcode = i[-7:]
#     postcode_area = postcode.split(" ")[0]
#     if postcode_area == 'n':
#         postcode_area = postcode.split(" ")[1]


#     try:
#         prop_type = v['property_type']
#         num_bedrooms = v["bedrooms"]
#     except KeyError:
#         continue
#     else:
#         for a, b in houses1.items():
#             postcode = i[-8:]
#             if postcode[0] == " ":
#                 postcode = i[-7:]
#             postcode_area1 = postcode.split(" ")[0]
#             if postcode_area1 == 'n':
#                 postcode_area1 = postcode.split(" ")[1]
#             if postcode_area == postcode_area1:
#                 try:
#                     prop_type1 = b['property_type']
#                     num_bedrooms1 = b['bedrooms']
#                 except KeyError:
#                     pass
#                 else:
#                     if prop_type1 == prop_type and num_bedrooms1 == num_bedrooms:
#                         local_similar_prices.append(b['price'])
    
#     avg_local_price = sum(local_similar_prices) / len(local_similar_prices)
#     v["avg_local_price"] = avg_local_price

In [ ]:
houses1AB

In [ ]:
# Need to convert the data into a quantitative format, which regression algorithms can learn/predict from
# This is done for each data set
properties = {}
index = 0
# loop through all of the 
for h, i in houses1AB.items():
    house_data = []
    
    # Area code has been commented out after speaking to Adam
    # area_code - Want to know which codes
    postcode = h[-8:]
    if postcode[0] == " ":
        postcode = h[-7:]
    postcode_area = postcode.split(" ")[0]
    if postcode_area == 'n':
        postcode_area = postcode.split(" ")[1]
    house_data.append(postcode_area)


    
    if 'latitude' and 'longitude' in i.keys():
        house_data.append(i['latitude'])
        house_data.append(i['longitude'])
    else:
        house_data.append("Na")
        house_data.append("Na")
    
    
    if i['property_type'] == 'Flat':
        prop_type = 1
    elif i['property_type'] == 'Terraced':
        prop_type = 2
    elif i['property_type'] == 'Semi-Detached':
        prop_type = 3 
    elif i['property_type'] == 'Detached':
        prop_type = 4
    else:
        prop_type = 5
    house_data.append(prop_type)
    
    #house_data.append(i["NLP_Score"])
    
    # Convert the data into the necessary string format
    date_obj = datetime.datetime.strptime(i['date'], "%d %b %Y")
    date_str = date_obj.strftime("%Y-%m-%d")
    house_data.append(date_str)
    
    if 'station_proximities' in i.keys() and i['station_proximities'] != {}:
        station_distance = min(i['station_proximities'].values())
        house_data.append(station_distance)
    else:
        house_data.append("NA")
        
    # You should have handled for the property_size difficulties by now
    if 'property_size' not in i.keys():
        house_data.append("NA")
    elif i['property_size'] == {} or i['property_size'] == 'Na' or i["property_size"] == None:
        house_data.append("NA")
    else:
        # This should return the sq foot of the property
        house_data.append(i["property_size"])
    
    # Add the bedrooms and bathrooms data   
    if "bedrooms" not in i.keys():
        house_data.append("NA")
    elif i['bedrooms'] == None or i['bedrooms'] == '':
        house_data.append("NA")
    else:
        house_data.append(i["bedrooms"])
        
    if 'bathrooms' not in i.keys():
        house_data.append("NA")
    elif i['bathrooms'] == None or i['bathrooms'] == '':
        house_data.append("NA")
    else:
        house_data.append(i["bathrooms"])
        
        
    # New build
    if 'new_build' not in i.keys():
        house_data.append("NA")
    elif i['new_build'] == '':
        house_data.append("NA")
    else:
        if i["new_build"]:
            house_data.append(1)
        else:
            house_data.append(0)
    
    if 'nearest_outstanding_school' not in i.keys():
        house_data.append("NA")
    else:
        house_data.append(i["nearest_outstanding_school"])
    
    #if 'avg_local_price' not in i.keys():
     #   house_data.append("NA")
    #else:
     #   house_data.append(i["avg_local_price"])
        
    if 'floorplan_url' in i.keys():
        house_data.append("Yes")
    else:
        house_data.append("NA")
        
    
    
    house_data.append(i["price"])
    
    
        
    properties[h] = house_data
    index += 1
    

In [ ]:
properties

# Now I build the Regression Model 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
properties_df = pd.DataFrame(properties)
properties_df = properties_df.T
properties_df.columns = ["Postcode_Area", "Latitude", "Longitude", "Property Type","Sold_Date", "Closest Station", "Size", "Bedrooms", "Bathrooms", "New Build", "Nearest OS", "Floorplan?", "Price"]
# Set the null values to numpy  NaN values
properties_df = properties_df.replace("NA", np.NaN)
properties_df = properties_df.replace("Na", np.NaN)
properties_df = properties_df.replace("na", np.NaN)

In [ ]:
properties_df

In [ ]:
properties_df.isnull().sum(axis = 0)

In [ ]:
postcode_similar_houses = properties_df.groupby(["Postcode_Area", "Property Type", "Bedrooms"])

In [ ]:
# Here is the median value for each postocode area, each number of bedrooms
local_similar_prices = postcode_similar_houses['Price'].median()


In [ ]:
properties_df['Local_Similar_Prices'] = np.NaN
# Now I need to get these median values into the properties_df
for i in properties_df.iterrows():
    try:
        local_similar_price = local_similar_prices[i[1][0], i[1][3], i[1][7]]
        properties_df.loc[i[0], "Local_Similar_Prices"] = local_similar_price
    except KeyError:
        continue

In [ ]:
properties_df

## Drop NaN's, using the local_similar_prices as the local price variable

In [ ]:
prop_df = properties_df
# Remove the columns which we don't need
del prop_df['Floorplan?']
del prop_df['Sold_Date']
del prop_df['Postcode_Area']
# Here we've dropped all of the rows without all of the data for the prop_df
prop_df = properties_df.dropna()

In [ ]:
prop_df

In [ ]:
x = prop_df.drop(['Price'], axis=1).values
y = prop_df['Price'].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [ ]:
# split the data set into a training and testing set
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
# Traing the model (on the training set)
ml = LinearRegression()
ml.fit(x_train,y_train)

In [ ]:
# Predict the test set results
y_pred = ml.predict(x_test)
print(y_pred)

In [ ]:
# evaluate the model 
from sklearn.metrics import r2_score, mean_absolute_error
print(r2_score(y_test, y_pred))

print(mean_absolute_error(y_test, y_pred))

# Now we're going to use the KNN approach to finding nearby house prices, instead of the postcode approach

In [ ]:
# Now we can try to do our KNN
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from sklearn.model_selection import cross_val_predict

In [ ]:
houses1AB

In [ ]:
knn_data1AB = {}
for i,v in houses1AB.items():
    try:
        knn_data1AB[i] = [v['latitude'], v['longitude'], v['price']]
    except KeyError:
        pass
knn_df1AB = pd.DataFrame(knn_data1AB)
knn_df1AB = knn_df1AB.T
knn_df1AB.columns = ["Latitude", "Longitude", "Price"]

In [ ]:
knn_df1AB

In [ ]:
knn_df1AB = knn_df1AB.dropna()

In [ ]:
x_1 = knn_df1AB[["Latitude", "Longitude"]]
y_1 = knn_df1AB["Price"]

In [ ]:
knn = KNeighborsRegressor(n_neighbors=100)
# Using a cross validation of 5 folds
y_pred_1 = cross_val_predict(knn, x_1, y_1, cv=5)

In [ ]:
# Our predictions in an array 
y_pred_1

In [ ]:
# Evaluation of the KNN1 predictions
print(r2_score(y_1, y_pred_1))
print(mean_absolute_error(y_1, y_pred_1))

In [ ]:
# I think it's a good idea improve the KNN is to include features of the 
# property, not just its  price. 
# Let's include the following features: lat, lon, property type, 
# And then after we can include bedrooms and bathrooms 
# to see if this improves performance.

In [ ]:
knn_df2 = properties_df[['Latitude', "Longitude", 'Property Type', 'Price']]
knn_df2 = knn_df2.dropna()

In [ ]:
x2 = knn_df2[['Latitude', "Longitude", 'Property Type']]
y2 = knn_df2['Price']

In [ ]:
knn2 = KNeighborsRegressor(n_neighbors=100)
# Using a cross validation of 5 folds
y_pred2 = cross_val_predict(knn2, x2, y2, cv=5)

In [ ]:
# Evaluation of the KNN2 predictions
print(r2_score(y2, y_pred2))
print(mean_absolute_error(y2, y_pred2))

In [ ]:
# That's a clear improvement, so now let's have a look at when you include 
# bedrooms and bathrooms

In [ ]:
knn_df3 = properties_df[['Latitude', "Longitude", 'Property Type',
                         'Price', 'Bedrooms', 'Bathrooms']]
knn_df3 = knn_df3.dropna()
x3 = knn_df3[['Latitude', "Longitude", 'Property Type', 'Bedrooms', "Bathrooms"]]
y3 = knn_df3['Price']

knn3 = KNeighborsRegressor(n_neighbors=19)
# Using a cross validation of 5 folds
y_pred3 = cross_val_predict(knn3, x3, y3, cv=5)
# Evaluation of the KNN2 predictions
print(r2_score(y3, y_pred3))
print(mean_absolute_error(y3, y_pred3))

In [ ]:
# This is an even larger improvement. So, this is what we want to use for the local similar prices (KNN).
# So, let's add the KNN predictions to our knn3 dataframe
knn_df3['KNN'] = y_pred3
knn_df3

In [ ]:
# Now we want to add this local similar prices (KNN) column to the 
prop_dfknn = properties_df
#del prop_dfknn['Local_Similar_Prices']
prop_dfknn["local_prices_knn"] = np.NaN

In [ ]:
prop_dfknn

In [ ]:
# Now I need to get these median values into the properties_df
for i in knn_df3.iterrows():
    prop_dfknn.loc[i[0], "local_prices_knn"] = i[1][6]
prop_dfknn = prop_dfknn.dropna()


In [ ]:
prop_dfknn

In [ ]:
x_knn = prop_dfknn.drop(['Price'], axis=1).values
y_knn = prop_dfknn['Price'].values

In [ ]:
# split the data set into a training and testing set
x_train_knn,x_test_knn,y_train_knn,y_test_knn = train_test_split(x_knn,y_knn,test_size=0.3,random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
# Traing the model (on the training set)
ml_knn = LinearRegression()
ml_knn.fit(x_train_knn,y_train_knn)

In [ ]:
# Predict the test set results
y_pred_knn = ml.predict(x_test_knn)
print(y_pred_knn)

In [ ]:
# evaluate the model 
from sklearn.metrics import r2_score, mean_absolute_error
print(r2_score(y_test_knn, y_pred_knn))

print(mean_absolute_error(y_test_knn, y_pred_knn))